# COVID-19 Data Cleaning Script

### Imports

In [237]:
import pandas as pd
import numpy as np

### Loading Data

In [238]:
# Load the CSV
df = pd.read_csv("Covid Data.csv")

### Print functions to understand data

In [239]:
def printColumnValueCounts(df):
    for col in df.columns:
        print(f"Value counts for '{col}':")
        print(df[col].value_counts(dropna=False))
        print("-" * 50)  # Adds a separator for readability
        
def summarize_dataframe(df, num_unique_threshold=20):
    """
    For each column in the DataFrame:
      - If the column is numeric (dtype is int/float) and has more than `num_unique_threshold`
        unique (non-null) values, compute its mean and standard deviation.
      - Otherwise, treat it as categorical/boolean and compute the percentage breakdown
        of its values (including NaNs).
      
      Additionally, for every column, include the raw value counts (with NaNs included).
      
    Returns:
      A dictionary with column names as keys and a summary dictionary as values.
    """
    summary = {}
    
    for col in df.columns:
        col_summary = {}
        
        # Compute value counts (including NaN)
        counts = df[col].value_counts(dropna=False)
        counts_dict = {}
        for key, value in counts.items():
            # Convert NaN key to a string for clarity
            if pd.isna(key):
                counts_dict['NaN'] = value
            else:
                counts_dict[key] = value
        col_summary['Value Counts'] = counts_dict
        
        # Also compute the percentage breakdown of values (including NaN)
        total = counts.sum()
        perc_breakdown = (counts / total * 100).round(2)
        perc_dict = {}
        for key, value in perc_breakdown.items():
            if pd.isna(key):
                perc_dict['NaN'] = f"{value}%"
            else:
                perc_dict[key] = f"{value}%"
        
        # Determine if we treat the column as numeric or categorical
        num_unique = df[col].nunique(dropna=True)
        if pd.api.types.is_numeric_dtype(df[col]) and num_unique > num_unique_threshold:
            col_summary['Type'] = 'Numeric'
            col_summary['Mean'] = df[col].mean(skipna=True)
            col_summary['Standard Deviation'] = df[col].std(skipna=True)
        else:
            col_summary['Type'] = 'Categorical'
            col_summary['Percentage Breakdown'] = perc_dict
        
        summary[col] = col_summary
        
    return summary

def printSummary(summaryDict):
    for col, summ in summaryDict.items():
      print(f"Summary for '{col}':")
      print("  Value Counts:")
      for val, count in summ['Value Counts'].items():
          print(f"    {val}: {count}")
      
      print(f"  Type: {summ['Type']}")
      if summ['Type'] == 'Numeric':
          print(f"  Mean: {summ['Mean']:.2f}")
          print(f"  Standard Deviation: {summ['Standard Deviation']:.2f}")
      else:
          print("  Percentage Breakdown:")
          for val, perc in summ['Percentage Breakdown'].items():
              print(f"    {val}: {perc}")
      print("-" * 50)

In [240]:
printColumnValueCounts(df)

Value counts for 'USMER':
USMER
2    662903
1    385672
Name: count, dtype: int64
--------------------------------------------------
Value counts for 'MEDICAL_UNIT':
MEDICAL_UNIT
12    602995
4     314405
6      40584
9      38116
3      19175
8      10399
10      7873
5       7244
11      5577
13       996
7        891
2        169
1        151
Name: count, dtype: int64
--------------------------------------------------
Value counts for 'SEX':
SEX
1    525064
2    523511
Name: count, dtype: int64
--------------------------------------------------
Value counts for 'PATIENT_TYPE':
PATIENT_TYPE
1    848544
2    200031
Name: count, dtype: int64
--------------------------------------------------
Value counts for 'DATE_DIED':
DATE_DIED
9999-99-99    971633
06/07/2020      1000
07/07/2020       996
13/07/2020       990
16/06/2020       979
               ...  
24/11/2020         1
17/12/2020         1
08/12/2020         1
16/03/2021         1
22/04/2021         1
Name: count, Length: 401, dt

### Cleaning Columns
In the Boolean features, 1 means "yes" and 2 means "no". values as 97-99 are missing data.

In boolean categorical columns, 97 - 98 is unknown, so we set those to NAN.

For PREGNANT, all males should have NAN, and then we should not allow women to be unknown, so we set then to -1 to make them as invalid. rows that contain -1 will be dropped later. 

DATE_DIED get's remapped to be DIED. If a person has a invalid date of death (9999-99-99), they did not die. 

PNEUMONIA is an output column. It cannot have unknown values. Mark those as -1 to be dropped later. 

Dataset specified that if CLASIFFICATION_FINAL was higher than 3, it meant a patient did not have COVID.

In [241]:
# Columns where 97, 98, and 99 should be set to NaN
columns_to_nan = [
    "ASTHMA", "CARDIOVASCULAR", "COPD", "DIABETES", "HIPERTENSION", "ICU", 
    "INTUBED", "INMSUPR", "OBESITY", "OTHER_DISEASE", "RENAL_CHRONIC", "TOBACCO"
]
df[columns_to_nan] = df[columns_to_nan].replace({97: np.nan, 98: np.nan, 99: np.nan})

# Handle 'PREGNANT' column
df.loc[df["SEX"] == 2, "PREGNANT"] = np.nan  # If male, set PREGNANT to NaN
df.loc[(df["SEX"] == 1) & (df["PREGNANT"].isin([97, 98])), "PREGNANT"] = -1  # If female and 97 or 98, set to -1

# Process 'DATE_DIED'
df["DATE_DIED"] = df["DATE_DIED"].apply(lambda x: 2 if x == "9999-99-99" else 1)

# Process 'PNEUMONIA'
df["PNEUMONIA"] = df["PNEUMONIA"].replace(99, -1)

# Process 'CLASIFFICATION_FINAL'
df["CLASIFFICATION_FINAL"] = df["CLASIFFICATION_FINAL"].apply(lambda x: x if x in [1, 2, 3] else 0)

In [242]:
printColumnValueCounts(df)

Value counts for 'USMER':
USMER
2    662903
1    385672
Name: count, dtype: int64
--------------------------------------------------
Value counts for 'MEDICAL_UNIT':
MEDICAL_UNIT
12    602995
4     314405
6      40584
9      38116
3      19175
8      10399
10      7873
5       7244
11      5577
13       996
7        891
2        169
1        151
Name: count, dtype: int64
--------------------------------------------------
Value counts for 'SEX':
SEX
1    525064
2    523511
Name: count, dtype: int64
--------------------------------------------------
Value counts for 'PATIENT_TYPE':
PATIENT_TYPE
1    848544
2    200031
Name: count, dtype: int64
--------------------------------------------------
Value counts for 'DATE_DIED':
DATE_DIED
2    971633
1     76942
Name: count, dtype: int64
--------------------------------------------------
Value counts for 'INTUBED':
INTUBED
NaN    855869
2.0    159050
1.0     33656
Name: count, dtype: int64
--------------------------------------------------
Val

### Removing any columns where we had invalid values


In [243]:
df = df[~df.isin([-1]).any(axis=1)]

In [244]:
printColumnValueCounts(df)

Value counts for 'USMER':
USMER
2    660373
1    368810
Name: count, dtype: int64
--------------------------------------------------
Value counts for 'MEDICAL_UNIT':
MEDICAL_UNIT
12    594028
4     307335
6      38564
9      37756
3      18762
8      10211
10      7867
5       7061
11      5571
7        890
13       828
2        159
1        151
Name: count, dtype: int64
--------------------------------------------------
Value counts for 'SEX':
SEX
2    516154
1    513029
Name: count, dtype: int64
--------------------------------------------------
Value counts for 'PATIENT_TYPE':
PATIENT_TYPE
1    834625
2    194558
Name: count, dtype: int64
--------------------------------------------------
Value counts for 'DATE_DIED':
DATE_DIED
2    953037
1     76146
Name: count, dtype: int64
--------------------------------------------------
Value counts for 'INTUBED':
INTUBED
NaN    836806
2.0    158768
1.0     33609
Name: count, dtype: int64
--------------------------------------------------
Val

### Creating new target column 

This turns the CLASIFFICATION_FINAL into a binary classification of COVID-19 PRESENCE

In [245]:
# Create 'COVID-19 PRESENCE' column based on 'COVID-19 SEVERITY'
df["COVID-19 PRESENCE"] = df["CLASIFFICATION_FINAL"].map({1: 1, 2: 1, 3: 1, 0: 2})

### Renaming Columns for Clarity

In [246]:
# Rename columns
df = df.rename(columns={
    "HIPERTENSION": "HYPERTENSION",
    "CLASIFFICATION_FINAL": "COVID-19 SEVERITY"
})

# List of output columns that should be at the end
output_columns = ["COVID-19 PRESENCE", "COVID-19 SEVERITY", "DATE_DIED", "PNEUMONIA"]

# Get all columns sorted alphabetically, excluding the output columns
sorted_columns = sorted([col for col in df.columns if col not in output_columns])

# Create the new column order with output columns at the end
new_column_order = sorted_columns + output_columns

# Reorder the DataFrame
df = df[new_column_order]

### Data Summarization

We view the data from the following subgroups:

Original Data

All Male Data

All Female Data

Non-pregnant Females

Pregnant Females

In [247]:
all_data_summary = summarize_dataframe(df)
printSummary(all_data_summary)

Summary for 'AGE':
  Value Counts:
    30: 26570
    31: 25534
    28: 24856
    29: 24705
    34: 24535
    32: 24507
    37: 24348
    35: 24316
    36: 24290
    33: 24219
    38: 24159
    27: 23911
    40: 23609
    39: 23504
    26: 22823
    46: 22413
    45: 22177
    41: 22133
    47: 22081
    43: 22079
    42: 21952
    44: 21869
    48: 21703
    25: 20668
    49: 20539
    50: 19913
    51: 18741
    52: 18233
    24: 17883
    53: 16921
    54: 16073
    55: 15952
    56: 15619
    23: 15017
    57: 14456
    58: 13584
    59: 13122
    22: 12469
    60: 12396
    21: 11073
    61: 10864
    63: 9757
    20: 9744
    62: 9704
    64: 8685
    65: 8566
    66: 7912
    19: 7808
    67: 7329
    68: 6902
    69: 6173
    70: 6155
    71: 5416
    72: 5285
    18: 5162
    73: 4872
    74: 4413
    17: 4323
    75: 4310
    1: 4168
    76: 3821
    0: 3811
    16: 3764
    77: 3728
    78: 3512
    15: 3323
    80: 2945
    79: 2932
    14: 2894
    2: 2717
    13: 2697
    

In [248]:
male_df = df[df["SEX"] == 2]
male_summary = summarize_dataframe(male_df)
printSummary(male_summary)

Summary for 'AGE':
  Value Counts:
    30: 12907
    31: 12471
    34: 12033
    35: 12007
    28: 12000
    29: 11995
    38: 11982
    37: 11948
    32: 11891
    33: 11779
    36: 11712
    40: 11652
    39: 11615
    27: 11466
    45: 11107
    41: 11081
    46: 11080
    43: 10980
    26: 10891
    44: 10818
    47: 10767
    42: 10702
    48: 10627
    49: 10004
    25: 9896
    50: 9826
    51: 9272
    52: 9108
    53: 8495
    24: 8390
    55: 8210
    54: 8157
    56: 8127
    57: 7525
    58: 7160
    59: 7114
    23: 7005
    60: 6612
    22: 6106
    61: 5923
    21: 5435
    63: 5291
    62: 5207
    64: 4762
    20: 4759
    65: 4745
    66: 4330
    67: 4082
    68: 3852
    19: 3650
    70: 3492
    69: 3428
    71: 3002
    72: 2920
    73: 2643
    18: 2540
    74: 2489
    1: 2373
    75: 2372
    76: 2159
    0: 2085
    77: 2069
    17: 2005
    78: 1909
    16: 1796
    15: 1714
    79: 1621
    80: 1583
    2: 1523
    14: 1440
    13: 1371
    81: 1350
    82: 

In [249]:
female_df = df[df["SEX"] == 1]
female_summary = summarize_dataframe(female_df)
printSummary(female_summary)

Summary for 'AGE':
  Value Counts:
    30: 13663
    31: 13063
    28: 12856
    29: 12710
    32: 12616
    36: 12578
    34: 12502
    27: 12445
    33: 12440
    37: 12400
    35: 12309
    38: 12177
    40: 11957
    26: 11932
    39: 11889
    46: 11333
    47: 11314
    42: 11250
    43: 11099
    48: 11076
    45: 11070
    41: 11052
    44: 11051
    25: 10772
    49: 10535
    50: 10087
    24: 9493
    51: 9469
    52: 9125
    53: 8426
    23: 8012
    54: 7916
    55: 7742
    56: 7492
    57: 6931
    58: 6424
    22: 6363
    59: 6008
    60: 5784
    21: 5638
    20: 4985
    61: 4941
    62: 4497
    63: 4466
    19: 4158
    64: 3923
    65: 3821
    66: 3582
    67: 3247
    68: 3050
    69: 2745
    70: 2663
    18: 2622
    71: 2414
    72: 2365
    17: 2318
    73: 2229
    16: 1968
    75: 1938
    74: 1924
    1: 1795
    0: 1726
    76: 1662
    77: 1659
    15: 1609
    78: 1603
    14: 1454
    80: 1362
    13: 1326
    79: 1311
    2: 1194
    12: 1188
    81

In [250]:
pregnant_female_df = female_df[female_df["PREGNANT"] == 1]
pregnant_female_summary = summarize_dataframe(pregnant_female_df)
printSummary(pregnant_female_summary)

Summary for 'AGE':
  Value Counts:
    27: 494
    28: 464
    30: 449
    26: 449
    29: 439
    25: 417
    31: 414
    32: 378
    24: 366
    23: 359
    33: 350
    22: 343
    21: 334
    34: 304
    20: 301
    35: 270
    19: 251
    36: 233
    18: 188
    37: 185
    17: 140
    38: 136
    39: 121
    40: 99
    16: 87
    41: 56
    42: 52
    15: 43
    43: 37
    44: 20
    45: 11
    14: 10
    47: 9
    46: 8
    48: 5
    0: 5
    52: 4
    59: 4
    66: 2
    54: 2
    1: 2
    55: 2
    57: 2
    12: 2
    87: 2
    61: 2
    65: 2
    49: 2
    51: 2
    3: 1
    72: 1
    11: 1
    13: 1
    71: 1
    76: 1
    73: 1
    50: 1
    53: 1
    80: 1
    89: 1
    56: 1
    63: 1
  Type: Numeric
  Mean: 28.09
  Standard Deviation: 6.76
--------------------------------------------------
Summary for 'ASTHMA':
  Value Counts:
    2.0: 7648
    1.0: 214
    NaN: 8
  Type: Categorical
  Percentage Breakdown:
    2.0: 97.18%
    1.0: 2.72%
    NaN: 0.1%
--------------------

In [251]:
not_pregnant_female_df = female_df[female_df["PREGNANT"] == 2]
not_pregnant_female_summary = summarize_dataframe(not_pregnant_female_df)
printSummary(not_pregnant_female_summary)

Summary for 'AGE':
  Value Counts:
    30: 13214
    31: 12649
    28: 12392
    36: 12345
    29: 12271
    32: 12238
    37: 12215
    34: 12198
    33: 12090
    38: 12041
    35: 12039
    27: 11951
    40: 11858
    39: 11768
    26: 11483
    46: 11325
    47: 11305
    42: 11198
    48: 11071
    43: 11062
    45: 11059
    44: 11031
    41: 10996
    49: 10533
    25: 10355
    50: 10086
    51: 9467
    24: 9127
    52: 9121
    53: 8425
    54: 7914
    55: 7740
    23: 7653
    56: 7491
    57: 6929
    58: 6424
    22: 6020
    59: 6004
    60: 5784
    21: 5304
    61: 4939
    20: 4684
    62: 4497
    63: 4465
    64: 3923
    19: 3907
    65: 3819
    66: 3580
    67: 3247
    68: 3050
    69: 2745
    70: 2663
    18: 2434
    71: 2413
    72: 2364
    73: 2228
    17: 2178
    75: 1938
    74: 1924
    16: 1881
    1: 1793
    0: 1721
    76: 1661
    77: 1659
    78: 1603
    15: 1566
    14: 1444
    80: 1361
    13: 1325
    79: 1311
    2: 1194
    12: 1186
    81

In [252]:
# # Save the Cleaned Data
# df.to_excel("Cleaned Covid Data.xlsx", index=False)

# Considering INTUBED as Prediction Column 

Due to INTUBED having high NAN values, we did not want to consider it as both a potential feature and prediction column. In order to due so, we must drop all rows where Intubed is NAN. This greately reduces our dataset size. Additionally, it removes all data points where the patient returned home (PATIENT_TYPE=1) leaving only data points where a patient was hospitalized (PATIENT_TYPE=2)

In [253]:
# List of output columns that should be at the end
output_columns = ["COVID-19 PRESENCE", "COVID-19 SEVERITY", "DATE_DIED", "INTUBED", "PNEUMONIA"]

# Get all columns sorted alphabetically, excluding the output columns
sorted_columns = sorted([col for col in df.columns if col not in output_columns])

# Create the new column order with output columns at the end
new_column_order = sorted_columns + output_columns

# Reorder the DataFrame
df = df[new_column_order]
df = df.dropna(subset=['INTUBED'])

### Data Summarization

We view the data from the following subgroups:

Original Data

All Male Data

All Female Data

Non-pregnant Females

Pregnant Females

In [254]:
all_data_summary = summarize_dataframe(df)
printSummary(all_data_summary)

Summary for 'AGE':
  Value Counts:
    56: 4498
    60: 4340
    59: 4281
    61: 4252
    57: 4249
    55: 4207
    52: 4198
    58: 4099
    50: 4026
    54: 4018
    63: 3999
    53: 3965
    65: 3950
    51: 3933
    62: 3881
    48: 3810
    66: 3780
    49: 3768
    64: 3697
    47: 3654
    67: 3559
    68: 3545
    46: 3541
    45: 3464
    70: 3289
    69: 3261
    44: 3096
    71: 2946
    72: 2942
    43: 2937
    42: 2783
    73: 2723
    41: 2675
    74: 2577
    40: 2556
    75: 2539
    0: 2452
    39: 2402
    38: 2326
    76: 2265
    37: 2193
    77: 2159
    78: 2128
    36: 2115
    35: 2099
    34: 1972
    33: 1888
    80: 1840
    30: 1785
    32: 1781
    31: 1765
    79: 1764
    1: 1689
    81: 1531
    29: 1514
    82: 1455
    28: 1447
    27: 1399
    83: 1341
    26: 1304
    84: 1219
    25: 1072
    85: 1071
    24: 896
    86: 887
    87: 834
    2: 799
    23: 736
    22: 709
    88: 694
    21: 626
    89: 552
    90: 548
    20: 545
    4: 543
    3:

In [255]:
male_df = df[df["SEX"] == 2]
male_summary = summarize_dataframe(male_df)
printSummary(male_summary)

Summary for 'AGE':
  Value Counts:
    56: 2808
    60: 2664
    52: 2658
    59: 2638
    55: 2620
    57: 2584
    61: 2577
    50: 2543
    58: 2539
    54: 2499
    53: 2487
    51: 2423
    63: 2391
    48: 2388
    65: 2375
    47: 2337
    49: 2307
    62: 2301
    64: 2254
    46: 2245
    66: 2236
    45: 2188
    67: 2147
    68: 2124
    44: 1987
    70: 1982
    69: 1954
    43: 1850
    41: 1731
    42: 1725
    71: 1722
    72: 1720
    40: 1603
    73: 1562
    74: 1514
    39: 1485
    75: 1471
    38: 1443
    76: 1357
    0: 1345
    37: 1339
    36: 1293
    35: 1252
    77: 1252
    78: 1208
    34: 1197
    33: 1106
    1: 1032
    80: 1031
    79: 1019
    32: 1019
    31: 1005
    30: 974
    81: 887
    82: 847
    29: 835
    83: 754
    28: 744
    27: 691
    84: 672
    26: 643
    85: 571
    25: 513
    86: 483
    2: 475
    87: 457
    24: 412
    88: 369
    23: 339
    22: 324
    4: 308
    89: 307
    21: 291
    3: 287
    90: 272
    5: 241
    20:

In [256]:
female_df = df[df["SEX"] == 1]
female_summary = summarize_dataframe(female_df)
printSummary(female_summary)

Summary for 'AGE':
  Value Counts:
    56: 1690
    60: 1676
    61: 1675
    57: 1665
    59: 1643
    63: 1608
    55: 1587
    62: 1580
    65: 1575
    58: 1560
    66: 1544
    52: 1540
    54: 1519
    51: 1510
    50: 1483
    53: 1478
    49: 1461
    64: 1443
    48: 1422
    68: 1421
    67: 1412
    47: 1317
    70: 1307
    69: 1307
    46: 1296
    45: 1276
    71: 1224
    72: 1222
    73: 1161
    44: 1109
    0: 1107
    43: 1087
    75: 1068
    74: 1063
    42: 1058
    40: 953
    41: 944
    78: 920
    39: 917
    76: 908
    77: 907
    38: 883
    37: 854
    35: 847
    36: 822
    30: 811
    80: 809
    33: 782
    34: 775
    32: 762
    31: 760
    79: 745
    27: 708
    28: 703
    29: 679
    26: 661
    1: 657
    81: 644
    82: 608
    83: 587
    25: 559
    84: 547
    85: 500
    24: 484
    86: 404
    23: 397
    22: 385
    87: 377
    21: 335
    20: 331
    88: 325
    2: 324
    90: 276
    19: 260
    89: 245
    4: 235
    3: 229
    18: 226

In [257]:
pregnant_female_df = female_df[female_df["PREGNANT"] == 1]
pregnant_female_summary = summarize_dataframe(pregnant_female_df)
printSummary(pregnant_female_summary)

Summary for 'AGE':
  Value Counts:
    26: 124
    30: 109
    27: 107
    22: 104
    31: 94
    29: 93
    25: 90
    28: 89
    33: 79
    21: 78
    24: 77
    20: 76
    32: 72
    23: 71
    35: 60
    19: 59
    34: 56
    37: 55
    36: 48
    17: 42
    38: 39
    18: 38
    39: 30
    16: 21
    40: 18
    43: 18
    41: 17
    42: 13
    15: 11
    44: 6
    45: 4
    1: 2
    47: 1
    89: 1
    80: 1
    12: 1
    14: 1
    55: 1
  Type: Numeric
  Mean: 27.96
  Standard Deviation: 6.83
--------------------------------------------------
Summary for 'ASTHMA':
  Value Counts:
    2.0: 1744
    1.0: 58
    NaN: 4
  Type: Categorical
  Percentage Breakdown:
    2.0: 96.57%
    1.0: 3.21%
    NaN: 0.22%
--------------------------------------------------
Summary for 'CARDIOVASCULAR':
  Value Counts:
    2.0: 1788
    1.0: 14
    NaN: 4
  Type: Categorical
  Percentage Breakdown:
    2.0: 99.0%
    1.0: 0.78%
    NaN: 0.22%
--------------------------------------------------
Summar

In [258]:
not_pregnant_female_df = female_df[female_df["PREGNANT"] == 2]
not_pregnant_female_summary = summarize_dataframe(not_pregnant_female_df)
printSummary(not_pregnant_female_summary)

Summary for 'AGE':
  Value Counts:
    56: 1690
    60: 1676
    61: 1675
    57: 1665
    59: 1643
    63: 1608
    55: 1586
    62: 1580
    65: 1575
    58: 1560
    66: 1544
    52: 1540
    54: 1519
    51: 1510
    50: 1483
    53: 1478
    49: 1461
    64: 1443
    48: 1422
    68: 1421
    67: 1412
    47: 1316
    70: 1307
    69: 1307
    46: 1296
    45: 1272
    71: 1224
    72: 1222
    73: 1161
    0: 1107
    44: 1103
    43: 1069
    75: 1068
    74: 1063
    42: 1045
    40: 935
    41: 927
    78: 920
    76: 908
    77: 907
    39: 887
    38: 844
    80: 808
    37: 799
    35: 787
    36: 774
    79: 745
    34: 719
    33: 703
    30: 702
    32: 690
    31: 666
    1: 655
    81: 644
    28: 614
    82: 608
    27: 601
    83: 587
    29: 586
    84: 547
    26: 537
    85: 500
    25: 469
    24: 407
    86: 404
    87: 377
    23: 326
    88: 325
    2: 324
    22: 281
    90: 276
    21: 257
    20: 255
    89: 244
    4: 235
    3: 229
    19: 201
    18: 188

In [259]:
# # Save the Cleaned Data
# df.to_excel("Cleaned Covid Data (Intubed as Prediction Col.).xlsx", index=False)